In [43]:
import numpy as np
import slideio
import cv2
import json
import matplotlib.path as mpltPath
import os
import warnings
import tifffile
import xml.etree.ElementTree as ET
import pandas as pd
from PIL import Image
from math import sqrt
import matplotlib.pyplot as plt
from glob import glob
from copy import copy
from tqdm.notebook import tqdm
from skimage.draw import polygon2mask
import warnings

# 경고 메시지를 무시하도록 설정s
warnings.filterwarnings("ignore")

In [45]:
xml_list=glob('../../data/OriginalData/dst_xml/*.xml')
whi_list=[f.replace('/dst_xml/', '/svs/') for f in xml_list]
whi_list=[f.replace('.xml', '.svs') for f in whi_list]
image_path='../../data/re-label segmentation/Image/'
mask_path='../../data/re-label segmentation/mask/'

def size_ratio(scene,img_size):
    width = scene.rect[2]
    height = scene.rect[3]
    ratio=0
    inverse_ratio=0
    img_width=0
    img_height=0
    if width>height:
        ratio=img_size/width
        inverse_ratio=width/img_size
        img_width=img_size
        img_height=height*ratio
    else:
        ratio=img_size/height
        inverse_ratio=height/img_size
        img_height=img_size
        img_width=width*ratio
        
    return int(img_width),int(img_height),inverse_ratio

def padd(image,value):
    height, width = image.shape[0:2]
    margin = [np.abs(height - width) // 2, np.abs(height - width) // 2]
    if np.abs(height-width) % 2 != 0:
        margin[0] += 1

    # 가로, 세로 가운데 부족한 쪽에 margin 추가
    if height < width:
        margin_list = [margin, [0, 0]]
    else:
        margin_list = [[0, 0], margin]

    # color 이미지일 경우 color 채널 margin 추가
    if len(image.shape) == 3:
        margin_list.append([0,0])

    # 이미지에 margin 추가
    output = np.pad(image, margin_list, mode='constant',constant_values=value)
    return output

In [46]:
for i in range(len(whi_list)):
    file_extension_index=whi_list[i].find('.svs')
    fileName=os.path.basename(os.path.splitext(whi_list[i])[0])
    slide = slideio.open_slide(whi_list[i], "SVS")
    scene = slide.get_scene(0)
    img_width,img_height,ratio=size_ratio(scene,512)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    slide_block = scene.read_block((0, 0, svsWidth, svsHeight),size=(int(img_width), int(img_height)))
    image=cv2.cvtColor(slide_block, cv2.COLOR_BGR2RGB)
    dst_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    cv2.imwrite(image_path+fileName+'.tiff', padd(image,255))
    
    xml_open=xml_list[i]
    polygon_xml = ET.parse(xml_open)
    root = polygon_xml.getroot()
    root=list(root)[0]
    root=list(root)
    imageCount=0
    image_shape=(img_height,img_width)
    for annotation_count in range(len(root)):
        root1 =list(root[annotation_count])[0]
        polygon=[]
        for polygon_count in root1.iter('Coordinate'): 
            polygon.append([int(float(polygon_count.attrib['Y'])*1/ratio),int(float(polygon_count.attrib['X'])*1/ratio)])
        mask=polygon2mask(image_shape,polygon)
        dst_mask=mask+dst_mask
    dst_mask=np.where(dst_mask>0,255,0)
    dst_mask=cv2.cvtColor(dst_mask.astype(np.uint8), cv2.COLOR_GRAY2RGB)
    cv2.imwrite(mask_path+fileName+'.tiff', padd(dst_mask,0))